In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

import seaborn as sns
from collections import Counter

In [3]:
with open('train.txt') as f:
    train_list = []
    for line in f:
        s_line = line.strip().split('\t')
        train_list.append(s_line)
train_list.sort(key=len, reverse=True)

In [4]:
labels_template_list = ['__label__NORMAL', '__label__INSULT', '__label__OBSCENITY', '__label__THREAT']

In [8]:
def build_labels_bin(labels_list, input_list):
    labels_bin = [input_list[0], input_list[-1]]
    for item in labels_list:
        if item not in input_list:
            labels_bin.append(0)
        else:
            labels_bin.append(1)
    return labels_bin

In [6]:
train_df_columns=['id', 'comment', 'normal', 'insult', 'obscenity', 'threat']

In [7]:
labels_dict = {'normal': '__label__NORMAL', 'insult': '__label__INSULT', 'obscenity': '__label__OBSCENITY', 'threat': '__label__THREAT'}

In [10]:
def build_train_df(input_list):
    train_list = []
    for item in input_list:
        binary_labels = build_labels_bin(labels_template_list, item)
        train_list.append(dict(zip(train_df_columns, binary_labels)))
    return pd.DataFrame(train_list)

In [11]:
train_df = build_train_df(train_list)

In [13]:
train_df.tail()

,id,comment,normal,insult,obscenity,threat
148770,99767,"а вы думаете что полиция об этом не знает, про...",1,0,0,0
148771,87456,"ну а что вы тут возмущаетесь , все ведь точно ...",1,0,0,0
148772,185614,затерли до дыр данный ролик сколько лет ему уж...,1,0,0,0
148773,86412,если проследят за пидором то накажут,0,1,0,0
148774,205906,редкий долбоёб,0,1,0,0


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148775 entries, 0 to 148774
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         148775 non-null  object
 1   comment    148775 non-null  object
 2   normal     148775 non-null  int64 
 3   insult     148775 non-null  int64 
 4   obscenity  148775 non-null  int64 
 5   threat     148775 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 6.8+ MB


In [16]:
train_df['id'] = train_df['id'].astype('int64')

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148775 entries, 0 to 148774
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         148775 non-null  int64 
 1   comment    148775 non-null  object
 2   normal     148775 non-null  int64 
 3   insult     148775 non-null  int64 
 4   obscenity  148775 non-null  int64 
 5   threat     148775 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 6.8+ MB


In [18]:
all_data_df = pd.read_csv('data.txt', sep='\t', header=None, names=['id', 'comment'])
#all_data_df.columns = ['id', 'comment']

In [20]:
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248290 entries, 0 to 248289
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       248290 non-null  int64 
 1   comment  248290 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.8+ MB


In [21]:
all_data_df.shape

(248290, 2)

In [23]:
all_data_df.head()

,id,comment
0,41127,дворника надо тоже уничтожить!
1,6812,"моя старшая неделю шипела, не принимала подкид..."
2,6256,полностью с вами согласна!
3,189636,"хоть ногу вверх, ничего не изменится"
4,99053,а что значит - левого ребенка?


In [24]:
test_id_df = pd.read_csv('test.txt', sep='\t', header=None, names=['id'])

In [26]:
test_id_df.head()

,id
0,167315
1,224546
2,241309
3,31170
4,173358


In [28]:
test_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99515 entries, 0 to 99514
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      99515 non-null  int64
dtypes: int64(1)
memory usage: 777.6 KB


In [29]:
test_id_df.shape

(99515, 1)

In [30]:
test_df = pd.merge(all_data_df, test_id_df, on='id')

In [31]:
print(all_data_df.shape, test_id_df.shape, train_df.shape, test_df.shape)

(248290, 2) (99515, 1) (148775, 6) (99515, 2)


In [ ]:
temp = train_df.groupby('label').count()['comment'].reset_index().sort_values(by='comment',ascending=False)
temp.style.background_gradient(cmap='Oranges')

In [ ]:
plt.figure(figsize=(12,6))\n",
    "sns.countplot(x='label',data=train_df)"

In [ ]:
fig = go.Figure(go.Funnelarea(\n",
    "    text = temp['label'],\n",
    "    values = temp['comment'],\n",
    "    title = {\"position\": \"top center\", \"text\": \"Funnel-Chart of Sentiment Distribution\"}\n",
    "    ))\n",
    "fig.show()

In [ ]:
from sklearn import preprocessing\n",
    "lb = preprocessing.LabelBinarizer()\n",
    "lb.fit(temp['label'])\n",
    "# LabelBinarizer()\n",
    "lb.classes_

In [ ]:
lb.transform(temp['label'])

In [ ]:
le = preprocessing.LabelEncoder()\n",
    "le.fit(temp['label'])\n",
    "le.classes_

In [ ]:
le_labels = le.transform(temp['label'])

In [ ]:
list(le.inverse_transform(le_labels))

In [ ]:
train_df['label_enc'] = le.transform(train_df['label'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()

In [ ]:
m = cv.fit_transform(train_df['comment'])

In [ ]:
m

In [ ]:
"#del train_df['comment']\n",
    "del train_df['label']"

In [ ]:
train_df['label_enc'].hist(bins=len(set(train_df['label_enc'])))

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree\n",
    "from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier\n",
    "from sklearn.model_selection import train_test_split"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(m, list(train_df['label_enc']))

In [ ]:
dtc = DecisionTreeClassifier(max_depth=10)

In [ ]:
rfc = RandomForestClassifier(n_jobs=-1)

In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
dtc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report

In [ ]:
preds = dtc.predict(X_test)

In [ ]:
print('Accuracy: ', float(accuracy_score(y_test, preds)*100), '%')\n",
    "print('Precision: ', precision_score(y_test, preds, average='macro'))\n",
    "print('Classification report: \\n', classification_report(y_test, preds))

In [ ]:
def calculate_metrics(self, submit):\n",
    "    df = self.answers.join(submit, on='id')\n",
    "\n",
    "    y_true = []\n",
    "    y_scores = []\n",
    "    for idx, row in df.iterrows():\n",
    "        answer_labels = row['labels'].split(',')\n",
    "        y_true.append([1 if label in answer_labels else 0 for label in LABELS])\n",
    "        y_scores.append([row['normal'], row['insult'], row['obscenity'], row['threat']])

In [ ]:
preds_r = rfc.predict(X_test)

In [ ]:
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 30,20
plot_tree(dtc, filled=True)